In [1]:
import datetime
import pandas as pd

import pickle

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
MODEL_FOLDER = "model"

PROCESSED_DATA_FOLDER = "data/4_all_data_preprocessed"
ISW_DATA_FILE = "all_isw.csv"
WEATHER_EVENTS_DATA_FILE = "features_generated_v1.csv"

OUTPUT_DATA_FILE = "all_features"

scaler_model = "scaler"
tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"
preciptype_encoder_model = "preciptype_encoder"
conditions_encoder_model = "conditions_encoder"

encoders_version = "v1"
scaler_version = "v1"
tfidf_transformer_version = "v4"
count_vectorizer_version = "v4"

In [4]:
def isNaN(num):
    return num != num

### Load preprocessed data

In [5]:
# Load CSVs
df_isw = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{ISW_DATA_FILE}", sep=";")
df_weather_events = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{WEATHER_EVENTS_DATA_FILE}", sep=";")

df_weather_events = df_weather_events.loc[:, ~df_weather_events.columns.str.contains('^Unnamed')]

print(df_isw.shape)

# Load TF-IDF
tfidf = pickle.load(open(f"{MODEL_FOLDER}/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
cv = pickle.load(open(f"{MODEL_FOLDER}/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

(402, 7)


### Prepare weather & events

In [6]:
# Technicaly it's feature engineering
print(df_weather_events['event_simultaneous_alarms'].max(), df_weather_events['event_simultaneous_alarms'].min())


58 0


In [7]:

# Replace timestamps with numerical values
df_weather_events.day_sunset = df_weather_events.day_sunset.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.day_sunrise = df_weather_events.day_sunrise.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.hour_datetime = df_weather_events.hour_datetime.apply(lambda x:
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()//3600
)

# Encode categories & save encoder
le = preprocessing.LabelEncoder()

df_weather_events.hour_conditions = le.fit_transform(df_weather_events.hour_conditions)
with open(f"{MODEL_FOLDER}/{conditions_encoder_model}_{encoders_version}.pkl", 'wb') as handle:
    pickle.dump(le, handle)

df_weather_events.hour_preciptype = le.fit_transform(df_weather_events.hour_preciptype)
with open(f"{MODEL_FOLDER}/{preciptype_encoder_model}_{encoders_version}.pkl", 'wb') as handle:
    pickle.dump(le, handle)

# Create new column with boolean event indicator
df_weather_events["event_indicator"] = df_weather_events.event_hour_level_event_datetimeEpoch.apply(lambda x: not isNaN(x))

# Drop now irrelevant features
features_to_drop = [
    "day_datetimeEpoch",
    "hour_datetimeEpoch",
    "event_all_region",
    "event_clean_end",
    "event_intersection_alarm_id",
    "event_start_time",
    "event_end_time",
    "event_start_hour",
    "event_end_hour",
    "event_day_date",
    "event_start_hour_datetimeEpoch",
    "event_end_hour_datetimeEpoch",	
    "event_hour_level_event_time",
    "event_hour_level_event_datetimeEpoch",
    "event_region_title",
    "event_region_city",
    "event_start",
    "event_end",
    "city",
    "region",
    "center_city_ua",
    "center_city_en",
    "region_alt"
]
df_weather_events = df_weather_events.drop(features_to_drop, axis=1)

# Convert everything to float
df_weather_events.hour_preciptype = df_weather_events.hour_preciptype.astype(float)
df_weather_events.hour_conditions = df_weather_events.hour_conditions.astype(float)
df_weather_events.region_id = df_weather_events.region_id.astype(float)
df_weather_events.event_alarms_past_24 = df_weather_events.event_alarms_past_24.astype(float)
df_weather_events.event_simultaneous_alarms = df_weather_events.event_simultaneous_alarms.astype(float)

df_weather_events.event_holiday_is_near = df_weather_events.event_holiday_is_near.apply(lambda x: 1.0 if x and not isNaN(x) else 0.0)

# Fill NAN
df_weather_events.fillna(0, inplace=True)

print(df_weather_events.shape)
print(df_weather_events.dtypes)
df_weather_events.sample(10)

(195084, 41)
city_resolvedAddress            object
day_datetime                    object
day_tempmax                    float64
day_tempmin                    float64
day_temp                       float64
day_dew                        float64
day_humidity                   float64
day_precip                     float64
day_precipcover                float64
day_solarradiation             float64
day_solarenergy                float64
day_uvindex                    float64
day_sunrise                    float64
day_sunset                     float64
day_moonphase                  float64
hour_datetime                  float64
hour_temp                      float64
hour_humidity                  float64
hour_dew                       float64
hour_precip                    float64
hour_precipprob                float64
hour_snow                      float64
hour_snowdepth                 float64
hour_preciptype                float64
hour_windgust                  float64
hour_windspe

,city_resolvedAddress,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_sunrise,day_sunset,day_moonphase,hour_datetime,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,region_id,event_alarms_past_24,event_simultaneous_alarms,event_hours_from_last_alarm,event_holiday_is_near,event_indicator
177142,"Черкаси, Україна",2022-11-14,7.9,3.7,5.7,3.5,86.0,0.000,0.00,80.9,7.0,4.0,25247.0,58259.0,0.66,14.0,7.4,71.58,2.6,0.0,0.0,0.0,0.0,5.0,18.4,11.2,356.9,1028.0,24.1,36.0,273.0,1.0,3.0,10.0,4.0,23.0,1.0,0.0,13.113889,0.0,False
169142,"Івано-Франківськ, Україна",2022-11-28,2.0,-0.2,0.5,-0.7,91.6,0.000,0.00,47.0,4.0,3.0,28160.0,59309.0,0.13,3.0,0.2,98.56,0.0,0.0,0.0,0.0,3.1,5.0,2.9,1.8,115.6,1029.0,24.1,96.1,0.0,0.0,0.0,10.0,3.0,9.0,0.0,0.0,11.791667,0.0,False
57736,"Чернігів, Україна",2022-08-29,30.1,15.4,21.6,15.8,73.8,0.200,8.33,192.0,16.6,7.0,21737.0,71298.0,0.05,13.0,25.4,60.86,17.3,0.0,0.0,0.0,0.0,5.0,26.6,13.0,255.9,1012.0,24.1,0.0,705.0,2.5,7.0,10.0,0.0,25.0,1.0,1.0,17.097500,0.0,False
154577,"Львів, Україна",2022-03-05,0.4,-1.9,-0.9,-4.3,78.1,0.400,16.67,92.2,6.7,3.0,25147.0,65554.0,0.07,18.0,-0.4,66.90,-5.8,0.0,0.0,0.0,0.0,5.0,20.2,7.6,24.5,1023.0,24.1,100.0,154.0,0.6,2.0,10.0,3.0,13.0,0.0,5.0,19.442222,0.0,False
67474,"Одеса, Україна",2022-10-25,17.3,14.0,15.7,14.1,90.1,1.600,12.50,93.5,8.1,4.0,26888.0,64407.0,0.98,9.0,15.2,93.15,14.1,0.0,0.0,0.0,0.0,5.0,25.9,14.8,179.6,1018.0,24.1,100.0,72.0,0.3,1.0,10.0,3.0,15.0,2.0,0.0,16.391667,0.0,False
174918,"Черкаси, Україна",2022-08-15,27.6,22.1,24.4,17.3,65.1,0.000,0.00,128.7,11.0,7.0,20554.0,72542.0,0.58,20.0,25.1,71.11,19.5,0.0,0.0,0.0,0.0,5.0,20.5,3.6,65.7,1008.0,24.1,100.0,22.0,0.1,0.0,3.0,3.0,23.0,0.0,0.0,3.766111,0.0,False
59639,"Чернігів, Україна",2022-11-16,4.7,0.5,3.5,2.3,91.4,1.700,50.00,13.3,1.1,1.0,26055.0,57861.0,0.73,1.0,4.6,96.56,4.1,0.0,0.0,0.0,0.0,5.0,8.3,4.3,246.5,1011.0,24.1,100.0,0.0,0.0,0.0,10.0,3.0,25.0,2.0,0.0,0.515556,0.0,True
170859,"Черкаси, Україна",2022-03-11,-3.6,-8.6,-6.1,-11.8,64.4,0.600,20.83,107.1,9.3,3.0,22449.0,64219.0,0.28,9.0,-8.1,72.91,-12.1,0.0,0.0,0.0,1.3,4.0,48.2,29.5,34.3,1029.0,0.4,100.0,148.0,0.5,1.0,10.0,3.0,23.0,7.0,0.0,8.512500,0.0,False
49847,"Чернівці, Україна",2022-09-18,13.8,8.3,11.4,8.6,83.7,20.534,8.33,30.9,2.7,1.0,25018.0,69787.0,0.73,12.0,9.3,86.54,7.2,0.0,0.0,0.0,0.0,3.0,39.6,31.6,279.0,999.8,16.8,99.6,81.0,0.3,1.0,10.0,3.0,24.0,0.0,5.0,18.397222,0.0,False
57974,"Чернігів, Україна",2022-09-08,17.5,2.9,10.9,5.2,70.0,0.000,0.00,140.8,12.0,7.0,22692.0,69955.0,0.39,11.0,14.2,67.18,8.2,0.0,0.0,0.0,0.0,5.0,14.4,6.5,349.9,1019.0,24.1,74.8,200.0,0.7,2.0,10.0,4.0,25.0,1.0,1.0,15.093611,1.0,False


### Calculate TF-IDF matrix

In [8]:
# Calculate word count vector for each report (save as matrix)
word_count_vector = cv.transform(df_isw["lemm"].values.astype("U"))
# print("Word count vector:\n", word_count_vector)

# Calculate global tfidf vectors matrix
tfidf_vector = tfidf.transform(word_count_vector)

# transform tfidf matrix to sparse dataframe
df_tfidf_vector = pd.DataFrame.sparse.from_spmatrix(tfidf_vector, columns=cv.get_feature_names_out())
df_tfidf_vector = pd.concat([df_isw.date_tomorrow_datetime, df_tfidf_vector], axis=1)

print(df_tfidf_vector.shape)

(402, 719)


### Merge final dataset

In [9]:
# Merge weather events dataset with yesterday report tfidf matrix (takes 2m to execute)
df_final = df_weather_events.merge(df_tfidf_vector.add_prefix("isw_"),
                                    how="inner",
                                    left_on="day_datetime",
                                    right_on="isw_date_tomorrow_datetime")

# Normalize
scaler = StandardScaler()
df_float_values = df_final.drop(['event_indicator', 'city_resolvedAddress', 'day_datetime', 'isw_date_tomorrow_datetime'], axis=1)
float_values_scaled = scaler.fit_transform(df_float_values)
df_fload_values_scaled = pd.DataFrame(float_values_scaled, columns=df_float_values.columns)
df_final = pd.concat([df_final[['event_indicator', 'city_resolvedAddress', 'day_datetime', 'isw_date_tomorrow_datetime']], df_fload_values_scaled], axis=1)

# Save scaler
with open(f"{MODEL_FOLDER}/{scaler_model}_{scaler_version}.pkl", 'wb') as handle:
    pickle.dump(scaler, handle)

print(df_final.shape)
df_final.sample(5)

c:\Users\tmyro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\tmyro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


(192867, 760)


,event_indicator,city_resolvedAddress,day_datetime,isw_date_tomorrow_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_sunrise,day_sunset,day_moonphase,hour_datetime,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,region_id,event_alarms_past_24,event_simultaneous_alarms,event_hours_from_last_alarm,event_holiday_is_near,isw_10km,isw_12km,isw_15km,isw_1st,isw_20km,isw_30km,isw_5km,isw_6km,isw_abandon,isw_abil,isw_abl,isw_access,isw_accus,isw_achiev,isw_across,isw_action,isw_activ,isw_ad,isw_addit,isw_address,isw_administr,isw_advanc,isw_advisor,isw_affect,isw_affili,isw_aim,isw_air,isw_airborn,isw_aircraft,isw_airstrik,isw_alexand,isw_allow,isw_almost,isw_along,isw_alongsid,isw_alreadi,isw_also,isw_although,isw_ammunit,isw_among,isw_amplifi,isw_annex,isw_announc,isw_anoth,isw_anti,isw_appar,isw_appeal,isw_appear,isw_appoint,isw_approxim,isw_april,isw_area,isw_arm,isw_armi,isw_armor,isw_around,isw_arrang,isw_arriv,isw_artilleri,isw_assault,isw_asset,isw_attempt,isw_author,isw_avail,isw_avdiivka,isw_avoid,isw_away,isw_axi,isw_back,isw_bakhmut,isw_bank,isw_base,isw_basi,isw_battalion,isw_battl,isw_becom,isw_began,isw_begin,isw_belaru,isw_belarusian,isw_belgorod,isw_bilohorivka,isw_blame,isw_border,isw_break,isw_bridg,isw_brigad,isw_build,isw_call,isw_campaign,isw_capabl,isw_captur,isw_carri,isw_case,isw_casualti,isw_cauldron,isw_caus,isw_center,isw_central,isw_challeng,isw_chang,isw_channel,isw_chechen,isw_chief,isw_child,isw_choos,isw_citizen,isw_civilian,isw_classifi,isw_clear,isw_click,isw_close,isw_colonel,isw_combat,isw_combin,isw_come,isw_command,isw_comment,isw_commerci,isw_commit,isw_commun,isw_compani,isw_complet,isw_compris,isw_concentr,isw_concern,isw_condemn,isw_condit,isw_confirm,isw_conflict,isw_connect,isw_conscript,isw_consid,isw_consist,isw_consolid,isw_construct,isw_contact,isw_contract,isw_control,isw_convent,isw_corp,isw_could,isw_council,isw_counter,isw_counterattack,isw_counteroffens,isw_countri,isw_cours,isw_cover,isw_creat,isw_crime,isw_crimea,isw_crimin,isw_critic,isw_cross,isw_current,isw_cut,isw_daili,isw_damag,isw_data,isw_date,isw_day,isw_decemb,isw_decis,isw_defend,isw_defens,isw_degrad,isw_demand,isw_demonstr,isw_deni,isw_deploy,isw_depot,isw_deputi,isw_describ,isw_despit,isw_destroy,isw_detail,isw_develop,isw_differ,isw_direct,isw_directli,isw_director,isw_discus,isw_disrupt,isw_district,isw_divis,isw_dmitri,isw_dnipro,isw_dnipropetrovsk,isw_dnr,isw_domest,isw_donba,isw_draw,isw_drive,isw_drone,isw_due,isw_duma,isw_earli,isw_east,isw_eastern,isw_econom,isw_effect,isw_eight,isw_eighteen,isw_eighti,isw_either,isw_element,isw_eleven,isw_emphas,isw_encircl,isw_end,isw_endnot,isw_energi,isw_enerhodar,isw_enforc,isw_engag,isw_enter,isw_entir,isw_entireti,isw_equip,isw_establish,isw_evacu,isw_evalu,isw_even,isw_evid,isw_exist,isw_expand,isw_expect,isw_explos,isw_express,isw_extens,isw_face,isw_facil,isw_fail,isw_failur,isw_fall,isw_fals,isw_famili,isw_far,isw_februari,isw_feder,isw_fifteen,isw_fifti,isw_fight,isw_fighter,isw_financi,isw_fire,isw_first,isw_five,isw_focu,isw_focus,isw_follow,isw_footag,isw_forcibl,isw_forecast,isw_foreign,isw_form,isw_format,isw_former,isw_forti,isw_fortif,isw_four,isw_fourteen,isw_front,isw_frontlin,isw_full,isw_fulli,isw_futur,isw_gain,isw_geneva,isw_geoloc,isw_geospati,isw_given,isw_gloc,isw_govern,isw_governor,isw_ground,isw_group,isw_grow,isw_guard,isw_gur,isw_haidai,isw_head,isw_heavi,isw_heavili,isw_held,isw_high,isw_highli,isw_highway,isw_himar,isw_hold,isw_hope,isw_hour,isw_howev,isw_hulyaipol,isw_human,isw_hundr,isw_hundred,isw_identifi,isw_imageri,isw_immedi,isw_immin,isw_import,isw_improv,isw_includ,isw_increas,isw_increasingli,isw_independ,isw_indic,isw_

In [10]:
# Save final dataset
with open(f"{PROCESSED_DATA_FOLDER}/{OUTPUT_DATA_FILE}.pkl", 'wb') as handle:
    pickle.dump(df_final, handle)